In [1]:
import torch

In [2]:
from src.nn.train import train, load_checkpoint
from src.dataset.dataset import SparkifyDataset

In [3]:
train_file = "data/train_dataset"
test_file = "data/test_dataset"

In [4]:
ds = SparkifyDataset(train_file)
ds_test = SparkifyDataset(test_file)

In [5]:
import json

In [6]:
unique_pages_file = "data/unique_pages.json"

In [7]:
with open(unique_pages_file, "r") as f:
    unique_pages = json.load(f)

In [8]:
target_dist_file = "data/target_dist.json"

In [9]:
with open(target_dist_file, "r") as f:
    target_dist = json.load(f)

In [10]:
total = 0
counts = []
for k, v in target_dist.items():
    total += v
    counts.append(v)

In [11]:
cross_entropy_weights = torch.tensor([total / c for c in counts])

In [12]:
from src.nn.Transformer import TransformerModel

In [13]:
model = TransformerModel(
    num_classes=len(unique_pages),
    nhead=8,
    num_layers=2
)

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [15]:
model = model.to(device)

In [16]:
from torch.optim.adam import Adam

In [17]:
opt = Adam(model.parameters(), lr=0.001)

In [18]:
target_epochs = 1000 * 1000
seen_epochs = 0
batch_size = 10
test_batch_size = 10
writer = "./runs/sparkify_transformer"
checkpoint_path = "./checkpoints/sparkify_transformer"

In [19]:
train(
    train_data=ds,
    test_data=ds_test,
    model=model,
    target_epochs=target_epochs,
    seen_epochs=seen_epochs,
    batch_size=batch_size,
    test_batch_size=test_batch_size,
    optm=opt,
    writer=writer,
    checkpoint_path=checkpoint_path,
    device=device,
    save_every_n=10000,
    cross_entropy_weights=cross_entropy_weights
)

  0%|          | 0/1000000 [00:00<?, ?it/s]

/run/media/john/Secondary/Documents/Cornell/Spring 2024/ORIE 4741 Learning From Data/Project/proj/lib64/python3.11/site-packages/torch/nested/__init__.py:166: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:177.)
  return _nested.nested_tensor(
